<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# AutoGen Prompt Chaining Agent

In this tutorial, we'll explore **prompt chaining** with [AutoGen agents](https://microsoft.github.io/autogen/0.2/docs/Getting-Started) and how to trace using Phoenix.
Prompt chaining is a method where a complex task is broken into smaller, linked subtasks, with the output of one step feeding into the next. This workflow is ideal when a task can be cleanly decomposed into fixed subtasks, making each LLM call simpler and more accurate — trading off latency for better overall performance.

AutoGen makes it easy to build these chains by coordinating multiple agents. Each `AssistantAgent` focuses on a specialized task, while a `UserProxyAgent` manages the conversation flow and passes key outputs between steps. With Phoenix tracing, we can visualize the entire sequence, monitor individual agent calls, and debug the chain more easily.

In this example, we'll create a multi-step market analysis workflow, from identifying trends to evaluating company strengths — all automated and fully traceable.

By the end of this tutorial, you’ll learn how to:

- Set up specialized AutoGen agents

- Chain tasks across agents with carryover context

- Improve task accuracy by simplifying each LLM call

- Trace and visualize the full execution using Phoenix

⚠️ You'll need an OpenAI Key for this tutorial.

## Set up Keys and Dependencies


In [ ]:
!pip install -qq pyautogen==0.9 autogen-agentchat~=0.2

In [ ]:
!pip install -qqq arize-phoenix arize-phoenix-otel openinference-instrumentation-openai

In [ ]:
import os
from getpass import getpass

import autogen

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

## Configure Tracing


In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="autogen-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

## Example Prompt Chaining Task: Market Analyzer

This example demonstrates how to use AutoGen agents to solve a chained series of market analysis tasks, while capturing detailed tracing with Phoenix. A main user agent delegates each task to a specialized expert agent.

Each task builds on the results of the previous one: after an agent completes its task, a summary of its output is passed as carryover context into the next agent's conversation. This ensures that every agent has access to relevant information gathered so far, allowing later steps to benefit from earlier research.

## Define Agent

First, we define four specialized `AssistantAgents`, each responsible for a different part of a market analysis workflow: analyzing market trends, identifying top-performing companies, comparing companies, and evaluating strengths and weaknesses. We also define a `UserProxyAgent` that acts as the coordinator, automatically sending tasks to the assistants and managing the overall conversation flow. The `human_input_mode` is set to "NEVER", meaning the user agent operates autonomously without waiting for manual input.

The `llm_config` specifies the configuration used for all the assistant agents.


![Agent Set up](https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_prompt_chaining.png)

In [ ]:
market_analysis_tasks = [
    """What is the latest market trend in the technology sector? Focus on freely accessible sources like news websites, research blogs, and online publications.""",
    """Based on the identified market trend, what are the top 5 companies currently leading in that sector? Focus on freely accessible sources like news websites, research blogs, and online publications.""",
    """Perform a comparison between these top 5 companies in terms of stock performance, revenue growth, and product innovation over the past year. Focus on freely accessible sources like news websites, research blogs, and online publications.""",
    """Summarize the strengths and weaknesses of these companies in terms of their market positioning and potential for future growth.""",
]

config_list = [
    {
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"],
    }
]

llm_config = {"config_list": config_list}

In [ ]:
market_trend_assistant = autogen.AssistantAgent(
    name="MarketTrendAssistant",
    llm_config=llm_config,
)

top_companies_assistant = autogen.AssistantAgent(
    name="TopCompaniesAssistant",
    llm_config=llm_config,
)

company_comparison_assistant = autogen.AssistantAgent(
    name="CompanyComparisonAssistant",
    llm_config=llm_config,
)

strength_weakness_assistant = autogen.AssistantAgent(
    name="StrengthWeaknessAssistant",
    llm_config=llm_config,
)

# User agent
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "")
    and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },
)

## Run Agent
Finally, we use `user.initiate_chats` to send each task to the appropriate specialized agent. Each agent receives its specific task, processes it, and produces a summary that can be carried over to the next step. The first task clears any prior conversation history, while the following tasks use reflection with an LLM to summarize and pass forward important context.

The AutoGen agent not only handles the task flow but also generates and executes code for desired outcomes.

In [ ]:
from opentelemetry.trace import StatusCode

tracer = tracer_provider.get_tracer(__name__)

# Initiating the chats
with tracer.start_as_current_span(
    "MarketAnalysisAgent",
    openinference_span_kind="agent",
) as agent_span:
    agent_span.set_status(StatusCode.OK)
    chat_results = user.initiate_chats(
        [
            {
                "recipient": market_trend_assistant,
                "message": market_analysis_tasks[0],
                "clear_history": True,
                "silent": False,
                "summary_method": "last_msg",
            },
            {
                "recipient": top_companies_assistant,
                "message": market_analysis_tasks[1],
                "summary_method": "reflection_with_llm",
            },
            {
                "recipient": company_comparison_assistant,
                "message": market_analysis_tasks[2],
                "summary_method": "reflection_with_llm",
            },
            {
                "recipient": strength_weakness_assistant,
                "message": market_analysis_tasks[3],
                "summary_method": "reflection_with_llm",
            },
        ]
    )

# View Results in Phoenix

In Phoenix, you can track how each LLM call in a prompt chain builds on previous outputs, with context from earlier steps guiding subsequent responses.

The tracing also shows the generated code AutoGen uses to fetch and process information, offering visibility into how the system interacts with external sources to complete tasks. This provides a clear view of the flow, from reasoning to code execution, ensuring accurate results.

You can also see metadata like latency and token usage. This will help you spot optimization opportunities to improve your agent’s efficiency.










![Results GIF](https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_prompt_chaining.gif)

Run the cell below to see full tracing results.

In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_prompt_chaining.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")